In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [28]:
df_full = pd.read_csv('/content/drive/MyDrive/df_full.csv')
df_full.sample(5)

,Price,Year,Mileage,City,State,Make,Model,Usage_level,City_imporatnce
498060,28440,2014,33957,OCONOMOWOC,WI,CHEVROLET,SILVERADO,Medium,Low
973747,29000,2015,41565,SCOTTSDALE,AZ,MERCEDES-BENZ,E-CLASSE350,Medium,Low
286151,11989,2012,112951,ATLANTA,GA,NISSAN,MAXIMA4DR,High,Low
1176186,37995,2015,33182,VICTORVILLE,CA,TOYOTA,TUNDRA,Medium,Low
1107330,13287,2016,41774,DRY RIDGE,KY,TOYOTA,CAMRYSE,Medium,Low


In [29]:
df_full = df_full[df_full.City_imporatnce != "Low"]

In [30]:
df_full.drop(columns=['City','State','City_imporatnce'],inplace=True)

In [31]:
df_full.sample(5)

,Price,Year,Mileage,Make,Model,Usage_level
371148,17995,2012,48408,HONDA,CROSSTOUREX-L,Medium
158011,38771,2015,37626,BMW,3,Medium
885335,11900,2015,30172,KIA,OPTIMALX,Medium
856308,10691,2014,61282,JEEP,COMPASSSPORT,Medium
608762,14890,2016,30913,FORD,ESCAPESE,Medium


In [32]:
X = df_full.iloc[:,1:]
y = df_full.iloc[:,0]

In [33]:
X.head()

,Year,Mileage,Make,Model,Usage_level
9,2015,13807,BUICK,ENCORECONVENIENCE,Low
10,2014,29535,BUICK,ENCOREFWD,Medium
18,2015,20681,BUICK,ENCORECONVENIENCE,Low
24,2015,21694,BUICK,ENCOREFWD,Low
26,2014,12760,BUICK,ENCOREFWD,Low


In [34]:
y.head()

9     17700
10    14555
18    15239
24    16400
26    16450
Name: Price, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y ,test_size=0.1,random_state=2)

In [36]:
from sklearn.ensemble import GradientBoostingRegressor

In [37]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [38]:
ohe=OneHotEncoder()
ohe.fit(X[['Make','Model','Usage_level']])

OneHotEncoder()

In [39]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Make','Model','Usage_level']),
                                    remainder='passthrough')

In [40]:
gbr_params = {'n_estimators': 500,
          'max_depth': 3,
          'min_samples_split': 5,
          'learning_rate': 1,
          'loss': 'squared_error'}

In [41]:
gbr = GradientBoostingRegressor(**gbr_params)

In [42]:
pipe=make_pipeline(column_trans,gbr)

In [43]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['ACURA', 'ALFA', 'AM', 'ASTON', 'AUDI', 'BENTLEY', 'BMW', 'BUICK',
       'CADILLAC', 'CHEVROLET', 'CHRYSLER', 'DODGE', 'FERRARI', 'FIAT',
       'FISKER', 'FORD', 'FREIGHTLINER', 'GENESIS', 'GMC', 'HONDA',
       'HUMMER', 'HYUNDAI', 'INFINITI', 'ISU...
       'PORSCHE', 'RAM', 'ROLLS-ROYCE', 'SAAB', 'SATURN', 'SCION',
       'SMART', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA', 'VOLKSWAGEN',
       'VOLVO'], dtype=object),
                                                                            array(['1', '124', '15002WD', ..., 'ZDXAWD', 'ZEPHYR4DR', 'ZX22DR'],
      dtype=object),
                                                                            array(['High', 'Low', 'Medium'], dtype=object)]),
                                     

In [44]:
df_full.sample(2)

,Price,Year,Mileage,Make,Model,Usage_level
793306,10999,2016,41321,HYUNDAI,ELANTRASE,Medium
272671,13500,2014,69035,NISSAN,MAXIMA3.5,Medium


In [45]:
y_pred=pipe.predict(X_test)

In [46]:
r2_score(y_test,y_pred)

0.9037666049675865

In [47]:
pipe.predict(pd.DataFrame(columns=['Year','Make','Model','Mileage','Usage_level'],data=np.array([7, 'KIA', 'OPTIMALX', 35279, 'Medium']).reshape(1,5)))

array([62227.68963594])

In [48]:
pipe.predict(pd.DataFrame(columns=['Year','Make','Model','Mileage','Usage_level'],data=np.array([7, 'TOYOTA', 'TACOMA4WD', 54258, 'Medium']).reshape(1,5)))

array([44293.77056961])

In [49]:
import pickle

In [50]:
pickle.dump(pipe,open('GBR.pkl','wb'))